In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
# ============================================================
# ✅ FIX PATH FOR NOTEBOOK INSIDE /notebooks
# ============================================================
import sys, os
from pathlib import Path

# Xác định thư mục cha của notebooks/
project_root = Path(os.getcwd()).parent  # => C:\Users\User\RR_model
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))  # ⚠️ thêm project_root, KHÔNG thêm src

print("📁 Project root added:", project_root)

# Kiểm tra lại
import importlib.util
print("🔍 src module found:", importlib.util.find_spec("src"))

📁 Project root added: C:\Users\User\RR_model
🔍 src module found: ModuleSpec(name='src', loader=<_frozen_importlib_external.SourceFileLoader object at 0x000002B154613DD0>, origin='C:\\Users\\User\\RR_model\\src\\__init__.py', submodule_search_locations=['C:\\Users\\User\\RR_model\\src'])


In [3]:
# ===========================================================
# 📘 Roll Rate Demo v7 - Sử dụng dữ liệu thật từ Oracle hoặc SQL file
# ===========================================================

from pathlib import Path
import sys
import pandas as pd
import numpy as np

# --- 1️⃣ Setup đường dẫn và import các module ---
root = Path(".").resolve()
sys.path.append(str(root / "src"))

from src.data_loader import load_data
from src.config import CFG, OUT_ROOT
from src.rollrate.transition import compute_transition_from_pairs, compute_transition_by_mob

from src.rollrate.export_excel import export_transition_excel
from src.rollrate.backtest import (
    compute_actual_matrix,
    rollforward_validation,
    plot_matrix_diff,
    plot_distribution_compare
)

print("✅ Import modules thành công.")
print("Cấu hình CFG:", CFG)


# --- 2️⃣ Load dữ liệu thực tế ---
#sql_path = r"D:\Python_code\RR_Model_1711\sql\uns2.sql"

#df = load_data(sql_path)
df = load_data()
df['DISBURSAL_DATE'] = df['DISBURSAL_DATE'].apply(lambda x: x.replace(day=1))

#print(f"📦 Đã load {len(df):,} dòng dữ liệu từ {sql_path}")

# Kiểm tra các cột quan trọng
required_cols = [CFG["loan"], CFG["mob"], CFG["state"], CFG["ead"], CFG["cutoff"]]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"❌ Thiếu các cột cần thiết trong data: {missing}")

print("✅ Dữ liệu có đủ các cột cần thiết:", required_cols)

# Nếu chưa có PRODUCT_TYPE thì thêm mặc định
if "PRODUCT_TYPE" not in df.columns:
    df["PRODUCT_TYPE"] = "A"
    print("⚠️ Không có PRODUCT_TYPE trong data, gán mặc định = 'A'")

# Convert cutoff về datetime
df[CFG["cutoff"]] = pd.to_datetime(df[CFG["cutoff"]])





✅ Import modules thành công.
Cấu hình CFG: {'loan': 'AGREEMENT_ID', 'mob': 'MOB', 'state': 'STATE_MODEL', 'orig_date': 'DISBURSAL_DATE', 'ead': 'PRINCIPLE_OUTSTANDING', 'disb': 'DISBURSAL_AMOUNT', 'cutoff': 'CUTOFF_DATE'}
📦 Loading Parquet from: C:\Users\User\RR_model\data\parquet
✅ Loaded 25,998,763 rows via pyarrow.dataset from C:\Users\User\RR_model\data\parquet


KeyboardInterrupt: 

In [ ]:
df['DISBURSAL_DATE'] = df['DISBURSAL_DATE'].apply(lambda x: x.replace(day=1))

#print(f"📦 Đã load {len(df):,} dòng dữ liệu từ {sql_path}")

# Kiểm tra các cột quan trọng
required_cols = [CFG["loan"], CFG["mob"], CFG["state"], CFG["ead"], CFG["cutoff"]]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"❌ Thiếu các cột cần thiết trong data: {missing}")

print("✅ Dữ liệu có đủ các cột cần thiết:", required_cols)

# Nếu chưa có PRODUCT_TYPE thì thêm mặc định
if "PRODUCT_TYPE" not in df.columns:
    df["PRODUCT_TYPE"] = "A"
    print("⚠️ Không có PRODUCT_TYPE trong data, gán mặc định = 'A'")

# Convert cutoff về datetime
df[CFG["cutoff"]] = pd.to_datetime(df[CFG["cutoff"]])

In [ ]:
from src.rollrate.transition import compute_transition_by_mob
from src.rollrate.export_excel import export_transition_excel
from src.config import OUT_ROOT

print("\n=== Tính transition matrices theo Product × MOB × Score ===")
matrices_by_mob, parent_fallback = compute_transition_by_mob(df)

# Đếm real vs fallback
count_real = 0
count_fb = 0

for prod, mob_dict in matrices_by_mob.items():
    for mob, score_dict in mob_dict.items():
        for score, obj in score_dict.items():
            if obj["is_fallback"]:
                count_fb += 1
            else:
                count_real += 1

print(f"🔹 Ma trận REAL     : {count_real}")
print(f"🔸 Ma trận FALLBACK : {count_fb}")
print(f"🔢 Tổng             : {count_real + count_fb}")

# Xuất Excel
export_transition_excel(
    matrices_by_mob,
    parent_fallback,
    out_path=OUT_ROOT / "Transition_ByProduct_MOB_Score_2.xlsx"
)


In [ ]:


export_transition_excel(
    matrices_by_mob,
    parent_fallback,
    out_path=OUT_ROOT / "Transition_ByProduct_MOB_Score_2.xlsx"
)


In [ ]:
# matrices_by_mob đã build từ module transition
# df = raw data chứa DISB_DATE, CUTOFF, MOB, PRINCIPLE_OUTSTANDING, STATE...

from src.rollrate.forecast import forecast_all_vintages


In [ ]:
forecast_results = forecast_all_vintages(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=29,
    enable_macro=False
)


In [ ]:
forecast_results

In [ ]:
from src.rollrate.lifecycle import (
    get_actual_all_vintages_amount,
    combine_all_lifecycle_amount,
    lifecycle_to_long_df_amount,
    build_full_lifecycle_amount,
    tag_forecast_rows_amount,
    add_del_metrics,
    aggregate_loss_to_product,
    aggregate_to_product,
    make_metric_pivot,
    export_lifecycle_all_products_one_file,
    aggregate_products_to_portfolio,
    extend_actual_info_with_portfolio
)


In [ ]:
# # 1) Build lifecycle actual + forecast
# df_lifecycle = build_full_lifecycle_amount(df, matrices_by_mob, max_mob=29)

# # 2) Tag forecast
# df_lifecycle_tagged = tag_forecast_rows_amount(df_lifecycle, df)

# # 3) DEL30/60/90
# df_del = add_del_metrics(df_lifecycle_tagged, df)

# # 4) Aggregate lên Product
# df_del_prod = aggregate_to_product(df_del)

# # 5) Pivot lifecycle DEL30+
# df_del30_pivot = df_del_prod.pivot_table(
#     index="VINTAGE_DATE",
#     columns="MOB",
#     values="DEL30_PCT"
# ).fillna(0)

# df_del30_pivot.index.name = "Cohort"

# # 6) Actual max MOB (để kẻ đường đỏ)
# actual_info = (
#     df.groupby("DISBURSAL_DATE")["MOB"]
#     .max()
#     .to_dict()
# )

# # 7) Export Excel đầy đủ format
# export_heatmap(
#     df_pivot=df_del30_pivot,
#     df_del_prod=df_del_prod,
#     actual_info=actual_info,
#     filename="DEL30P_Lifecycle.xlsx"
# )

# print("🎉 Lifecycle DEL30+ đã xuất thành công!")


In [ ]:
# # 1) Build lifecycle (actual + forecast, amount-based)
# df_lifecycle = build_full_lifecycle_amount(
#     df_raw=df,
#     matrices_by_mob=matrices_by_mob,
#     max_mob=29,
# )

# # 2) Tag forecast rows
# df_lifecycle_tagged = tag_forecast_rows_amount(
#     df_lifecycle=df_lifecycle,
#     df_raw=df,
# )

# # 3) Tính DEL30/60/90 amount + pct
# df_del = add_del_metrics(
#     df_lifecycle=df_lifecycle_tagged,
#     df_raw=df,
# )

# # 4) Aggregate Product_Score → Product
# df_del_prod = aggregate_to_product(df_del)

# # 5) Max actual MOB per (Product, Vintage)
# actual_info = (
#     df.groupby(["PRODUCT_TYPE", "DISBURSAL_DATE"])["MOB"]
#       .max()
#       .to_dict()
# )

# # 6) Export chung 1 file, nhiều sheet (mỗi Product × metric)
# export_lifecycle_all_products_one_file(
#     df_del_prod=df_del_prod,
#     actual_info=actual_info,
#     filename="Lifecycle_All_Products.xlsx",
# )


In [ ]:
# # 6) Export chung 1 file, nhiều sheet (mỗi Product × metric)
# export_lifecycle_all_products_one_file(
#     df_del_prod=df_del_prod,
#     actual_info=actual_info,
#     filename="Lifecycle_UnSecured.xlsx",
# )


## New pipeline


In [ ]:
# 1. Build lifecycle
df_lifecycle = build_full_lifecycle_amount(df, matrices_by_mob, max_mob=24)

# 2. Tag forecast rows
df_lifecycle = tag_forecast_rows_amount(df_lifecycle, df)

# 3. Add DEL metrics (level Product × Score × Vintage × MOB)
df_del = add_del_metrics(df_lifecycle, df)

# 4. Aggregate lên Product level
df_del_prod = aggregate_to_product(df_del)


In [ ]:
df_del_prod

In [ ]:
# 5. Aggregate lên portfolio
df_portfolio = aggregate_products_to_portfolio(
    df_del_prod,
    portfolio_name="PORTFOLIO_ALL"   # hoặc tên tuỳ bạn
)

# 6. Gộp lại: product + portfolio
df_export = pd.concat(
    [df_del_prod, df_portfolio],
    ignore_index=True
)


In [ ]:
# 5) Max actual MOB per (Product, Vintage)
actual_info = (
    df.groupby(["PRODUCT_TYPE", "DISBURSAL_DATE"])["MOB"]
      .max()
      .to_dict()
)

In [ ]:
# 5. Aggregate lên portfolio
df_portfolio = aggregate_products_to_portfolio(
    df_del_prod,
    portfolio_name="PORTFOLIO_ALL"   # hoặc tên tuỳ bạn
)

# 6. Gộp lại: product + portfolio
df_export = pd.concat(
    [df_del_prod, df_portfolio],
    ignore_index=True
)


In [ ]:
actual_info_extended = extend_actual_info_with_portfolio(
    actual_info_prod=actual_info,
    portfolio_name="PORTFOLIO_ALL"
)

export_lifecycle_all_products_one_file(
    df_del_prod=df_export,
    actual_info=actual_info_extended,
    filename="Lifecycle_All_Products_And_Portfolio.xlsx"
)


In [ ]:
export_lifecycle_all_products_one_file(
    df_del_prod=df_export,
    actual_info=actual_info_extended,
    filename="Lifecycle_All_Products_And_Portfolio.xlsx"
)

## sale plan

In [ ]:
from src.rollrate.forecast_plan import forecast_sale_plan_by_mob


In [ ]:
from src.rollrate.transition import compute_transition_by_mob, STATE_SPACE
from src.config import BUCKETS_CANON



sale_plan_fc = forecast_sale_plan_by_mob(
    sale_plan_df=sale_plan_df,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    mob_target=12,
    start_state=BUCKETS_CANON[0],
    states=STATE_SPACE,
)


In [ ]:
import pandas as pd
import numpy as np

from src.config import BUCKETS_CANON, ABSORBING_BASE
from src.rollrate.transition import compute_transition_by_mob, STATE_SPACE
from src.rollrate.forecast_plan import forecast_sale_plan_by_mob  # hàm bạn đặt trong forecast_plan.py


# ========================================
# 2) Load SALE PLAN + chuẩn hóa SCORE
# ========================================
sale_plan_df = pd.read_excel("C:/Users/User/RR_model/sale_plan_5yF2.xlsx", sheet_name="SALE_PLAN")
sale_plan_df["VINTAGE_DATE"] = pd.to_datetime(sale_plan_df["VINTAGE_DATE"])

print("✔ Sale Plan loaded:")
display(sale_plan_df.head())

# Chuẩn hóa RISK_SCORE
def normalize_score(x):
    if pd.isna(x):
        return "ALL"
    x_str = str(x).strip().upper()
    if x_str in ["", "NA", "NAN", "NONE", "NULL"]:
        return "ALL"
    return x_str

sale_plan_df["RISK_SCORE"] = sale_plan_df["RISK_SCORE"].apply(normalize_score)

print("✔ SCORE normalized:")
display(sale_plan_df["RISK_SCORE"].value_counts(dropna=False))

# ========================================
# 3) Forecast SALE PLAN
# ========================================
states = list(STATE_SPACE)  # dùng đúng state_space của engine

sale_plan_fc = forecast_sale_plan_by_mob(
    sale_plan_df=sale_plan_df,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    mob_target=24,
    start_state=BUCKETS_CANON[0],
    states=states,
)

print("✔ Forecast completed! Sample:")
display(sale_plan_fc.head(10))


## Report ALL

In [ ]:
from src.rollrate.forecast import forecast_all_vintages
from src.rollrate.lifecycle import (
    build_full_lifecycle_amount,
    tag_forecast_rows_amount,
    add_del_metrics,
    aggregate_to_product,
    aggregate_products_to_portfolio,
    export_lifecycle_all_products_one_file,
    extend_actual_info_with_portfolio,
)


In [ ]:
print("▶ Building lifecycle for EXISTING book...")

df_lc_existing = build_full_lifecycle_amount(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=24
)

df_lc_existing = tag_forecast_rows_amount(df_lc_existing, df)
df_lc_existing = add_del_metrics(df_lc_existing, df)

df_lc_existing["SOURCE"] = "EXISTING"


In [ ]:
df_plan = sale_plan_fc.copy()
df_plan["DISB_TOTAL"] = df_plan.groupby(
    ["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE"]
)["DPD0"].transform("first")


In [ ]:
# Load bucket groups từ module lifecycle (đã có sẵn trong file bạn gửi)
from src.rollrate.lifecycle import (
    BUCKETS_30P,
    BUCKETS_60P,
    BUCKETS_90P,
)

# Tính DEL metrics cho Sale Plan
df_plan["DEL30_AMT"] = df_plan[BUCKETS_30P].sum(axis=1)
df_plan["DEL60_AMT"] = df_plan[BUCKETS_60P].sum(axis=1)
df_plan["DEL90_AMT"] = df_plan[BUCKETS_90P].sum(axis=1)

df_plan["DEL30_PCT"] = df_plan["DEL30_AMT"] / df_plan["DISB_TOTAL"]
df_plan["DEL60_PCT"] = df_plan["DEL60_AMT"] / df_plan["DISB_TOTAL"]
df_plan["DEL90_PCT"] = df_plan["DEL90_AMT"] / df_plan["DISB_TOTAL"]
df_plan["IS_FORECAST"] = 1
df_plan["SOURCE"] = "SALE_PLAN"


In [ ]:
df_lifecycle_all = pd.concat(
    [df_lc_existing, df_plan],
    ignore_index=True
).sort_values(
    ["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE", "MOB"]
)

print("✔ GHÉP LIFECYCLE HOÀN TẤT")


In [ ]:
df_prod = aggregate_to_product(df_lifecycle_all)


In [ ]:
df_port = aggregate_products_to_portfolio(
    df_del_prod=df_prod,
    portfolio_name="PORTFOLIO_ALL",
)


In [ ]:
df_lifecycle_full = pd.concat([df_prod, df_port], ignore_index=True)


In [ ]:
# Actual info chỉ áp dụng lên EXISTING
actual_info_prod = (
    df_lc_existing[df_lc_existing["IS_FORECAST"] == 0]
    .groupby(["PRODUCT_TYPE", "VINTAGE_DATE"])["MOB"]
    .max()
    .to_dict()
)

# Extend sang portfolio
actual_info_all = extend_actual_info_with_portfolio(actual_info_prod)


In [ ]:
export_lifecycle_all_products_one_file(
    df_del_prod=df_lifecycle_full,
    actual_info=actual_info_all,
    filename="Lifecycle_All_Products_With_SalePlan.xlsx"
)


In [ ]:
import pandas as pd

# Giả sử DataFrame của bạn tên là df
df.to_parquet("output.parquet", compression="snappy", index=False)


## calibration

In [ ]:
from src.rollrate.lifecycle import (
    build_full_lifecycle_amount,
    tag_forecast_rows_amount,
    add_del_metrics,
)
from src.rollrate.calibration import (
    build_actual_lifecycle_amount_only,
    build_model_lifecycle_amount_only,
    compute_k_per_product_auto,
    apply_product_calibration,
    compute_month_seasonality,
    apply_month_seasonality,
)

# 1) Lifecycle full (actual + forecast Markov) – phục vụ báo cáo
df_lc_states = build_full_lifecycle_amount(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=24,
)

# Tag forecast rows
df_lc_states = tag_forecast_rows_amount(df_lc_states, df)

# Thêm DEL30/60/90, DISB_TOTAL
df_lc = add_del_metrics(df_lc_states, df)


In [ ]:
# 2) Lifecycle ACTUAL ONLY (cho backtest & month seasonality)
df_actual_only = build_actual_lifecycle_amount_only(df)

# 3) Lifecycle MODEL ONLY (forecast Markov cho các vintage lịch sử)
df_model_only = build_model_lifecycle_amount_only(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=24,
    enable_macro=False,
)


In [ ]:
# Chọn horizon backtest (MOB12 hoặc MOB9/18 tuỳ portfolio)
HORIZON = 12

k_dict = compute_k_per_product_auto(
    df_actual=df_actual_only,
    df_model=df_model_only,
    horizon_mob=HORIZON,
    metric_col="DEL90_PCT",
    min_cohort=5,
    clip_range=(0.5, 1.5),
)

print("k per product:", k_dict)


In [ ]:
month_factor = compute_month_seasonality(
    df_actual=df_actual_only,
    horizon_mob=HORIZON,
    metric_col="DEL90_PCT",
    min_cohort=5,
    clip_range=(0.7, 1.3),
)


In [ ]:
# 5.1 Apply k per product
df_lc_calib = apply_product_calibration(
    df_lifecycle=df_lc,
    k_dict=k_dict,
    metric_cols=("DEL30_PCT", "DEL60_PCT", "DEL90_PCT"),
    only_forecast=True,
)

# 5.2 Apply month seasonality
df_lc_calib = apply_month_seasonality(
    df_lifecycle=df_lc_calib,
    month_factor=month_factor,
    metric_cols=("DEL30_PCT", "DEL60_PCT", "DEL90_PCT"),
    only_forecast=True,
)


In [ ]:
# 1) Build lifecycle từ dữ liệu history
df_lifecycle = build_full_lifecycle_amount(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=29
)

# 2) Add DEL metrics
df_lifecycle = add_del_metrics(df_lifecycle, df)

# 3) Gắn cờ forecast
df_lifecycle = tag_forecast_rows_amount(df_lifecycle, df)

# 4) Tách actual vs forecast
df_actual   = df_lifecycle[df_lifecycle["IS_FORECAST"] == 0]
df_forecast = df_lifecycle[df_lifecycle["IS_FORECAST"] == 1]


In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
from src.rollrate.calibration import run_full_calibration

In [ ]:


df_lifecycle_calibrated = run_full_calibration(df_lifecycle_raw)


In [ ]:
from seasonality import build_seasonality

seasonality = build_seasonality(df_lifecycle, smooth_window=3)

print(seasonality)


In [ ]:
from seasonality import apply_seasonality_to_lifecycle

df_lifecycle_adj = apply_seasonality_to_lifecycle(df_lifecycle, seasonality)


In [ ]:
from seasonality import apply_seasonality_to_sale_plan

df_plan_fc_adj = apply_seasonality_to_sale_plan(df_plan_fc, seasonality)


In [ ]:
import pandas as pd
import numpy as np

from src.rollrate.transition import compute_transition_by_mob
from src.rollrate.forecast import forecast_all_vintages
from src.rollrate.forecast_plan import forecast_sale_plan_by_mob

from src.rollrate.calibration import (
    compute_k_per_product,
    apply_calibration_to_forecast,
    apply_calibration_to_sale_plan,
)

from src.rollrate.seasonality import (
    build_seasonality,
    apply_seasonality_to_lifecycle,
    apply_seasonality_to_sale_plan
)

from src.rollrate.lifecycle import (
    get_actual_all_vintages_amount,
    combine_all_lifecycle_amount,
    lifecycle_to_long_df_amount,
    tag_forecast_rows_amount,
    add_del_metrics,
    aggregate_to_product,
    aggregate_products_to_portfolio,
    extend_actual_info_with_portfolio,
    export_lifecycle_all_products_one_file,
)

from src.config import CFG, BUCKETS_CANON
